In [1]:
from casadi import SX, inv, vertcat, Function, integrator, sin, cos, diag

In [2]:
#Equations of motion expressed in BODY

#6 DOF states vectors
u = SX.sym('u') 
v = SX.sym('v') 
w = SX.sym('w')
v_nb = vertcat(u,v,w) #body-fixed linear velocity

p = SX.sym('p') 
q = SX.sym('q') 
r = SX.sym('r')
w_nb = vertcat(p,q,r) #body-fixed angular velocity

m = SX.sym('m')  # Mass

I_x = SX.sym('I_x') # moment of inertia x entry
I_y = SX.sym('I_y') # moment of inertia y entry
I_z = SX.sym('I_z') # moment of inertia z entry
I_zx = SX.sym('I_zx') # product of inertia zx entry 
I_yx = SX.sym('I_yx') # product of inertia yx entry 
I_zy = SX.sym('I_zy') # product of inertia zy entry 

x_g = SX.sym('x_g')  # Center of gravity, x-axis
y_g = SX.sym('y_g')  # Center of gravity, z-axis
z_g = SX.sym('z_g')  # Center of gravity, z-axis

x_b = SX.sym('x_b')  # Center of buoyancy, x-axis
y_b = SX.sym('y_b')  # Center of buoyancy, z-axis
z_b = SX.sym('z_b')  # Center of buoyancy, z-axis

X_du = SX.sym('X_du') # Added mass in surge
X_dw = SX.sym('X_dv') # coupled Added mass in surge & sway
X_dw = SX.sym('X_dw') # coupled Added mass in surge & heave
X_dp = SX.sym('X_dp') # coupled Added mass in surge & roll
X_dq = SX.sym('X_dq') # coupled Added mass in surge & pitch
X_dr = SX.sym('X_dr') # coupled Added mass in surge & pitch

Y_du = SX.sym('Y_du') # Added mass in sway & surge
Y_dv = SX.sym('Y_dv') # Added mass in sway
Y_dw = SX.sym('Y_dw') # coupled Added mass in sway & heave
Y_dp = SX.sym('Y_dp') # coupled Added mass in sway & roll
Y_dq = SX.sym('Y_dq') # coupled Added mass in sway & pitch
Y_dr = SX.sym('Y_dr') # coupled Added mass in sway & yaw

Z_du = SX.sym('Z_du') # Added mass in heave & surge
Z_dv = SX.sym('Z_dv') # Added mass in heave & sway
Z_dw = SX.sym('Z_dw') # coupled Added mass in heave
Z_dp = SX.sym('Z_dp') # coupled Added mass in heave & roll
Z_dq = SX.sym('Z_dq') # coupled Added mass in heave & pitch
Z_dr = SX.sym('Z_dr') # coupled Added mass in heave & yaw



In [15]:
# Rigid body kinetics

# body inertia tensor
I_rb = SX.sym('I_rb',3,3)
I_rb[0,0] = I_x
I_rb[1,1] = I_y
I_rb[2,2] = I_z
I_rb[1,0] = -I_yx
I_rb[2,0] = -I_zx
I_rb[2,1] = -I_zy
I_rb[0,1] = I_rb[1,0]
I_rb[0,2] = I_rb[2,0]
I_rb[1,2] = I_rb[2,1]

# Rigid-Body System Inertia Matrix
M_rb = SX(6, 6)  # Create a 6x6 symbolic matrix

# Manually populate the matrix
M_rb[:3, :3] = m*SX.eye(3)
M_rb[3:, 3:] = I_rb

S  =  SX.zeros(3,3) # inertia skew symmetric component
S[1,0] = m*z_g
S[2,0] = -m*y_g
S[2,1] = m*x_g
S[0,1] = -S[1,0]
S[0,2] = -S[2,0]
S[1,2] = -S[2,1]

M_rb[3:, :3] = S
M_rb[:3, 3:] = -S

# print(M_rb)
#assuming Starboard–port symmetrical underwater vehicles yg=0 and Ixy = Iyz = 0


# Coriolis-Centripetal Matrix from System Inertia Matrix
# Lagrangian Parameterization
C_rb = SX(6, 6)  # Create a 6x6 symbolic matrix
# Manually populate the matrix
c_m = SX(3, 3) 

c_m[0,0] = -m*(y_g*q + z_g*r)
c_m[1,0] = m*(x_g*q - w)
c_m[2,0] = m*(x_g*r + v)

c_m[0,1] = m*(y_g*p + w)
c_m[1,1] = -m*(z_g*r + x_g*p)
c_m[2,1] = m*(y_g*r - u)

c_m[0,2] = m*(z_g*p - v)
c_m[1,2] = m*(z_g*q + u)
c_m[2,2] = -m*(x_g*p + y_g*q)

c_I = SX.zeros(3,3)
c_I[1,0] = I_zy*q + I_zx*p - I_z*r
c_I[2,0] = -I_zy*r - I_yx*p + I_y*q
c_I[2,1] = I_zx*r + I_yx*q - I_x*p
c_I[0,1] = -c_I[1,0]
c_I[0,2] = -c_I[2,0]
c_I[1,2] = -c_I[2,1]

C_rb[:3, :3] = SX.zeros(3,3)
C_rb[:3, 3:] = -c_m.T
C_rb[3:, :3] = c_m
C_rb[3:, 3:] = c_I

C_rb
# Inertia Matrix ROV & AUV

SX(@1=0, @2=(m*((y_g*q)+(z_g*r))), @3=(m*((x_g*q)-w)), @4=(m*((x_g*r)+v)), @5=(m*((y_g*p)+w)), @6=(m*((z_g*r)+(x_g*p))), @7=(m*((y_g*r)-u)), @8=(m*((z_g*p)-v)), @9=(m*((z_g*q)+u)), @10=(m*((x_g*p)+(y_g*q))), @11=(((I_zy*q)+(I_zx*p))-(I_z*r)), @12=((I_y*q)-((I_zy*r)+(I_yx*p))), @13=(((I_zx*r)+(I_yx*q))-(I_x*p)), 
[[@1, @1, @1, @2, (-@3), (-@4)], 
 [@1, @1, @1, (-@5), @6, (-@7)], 
 [@1, @1, @1, (-@8), (-@9), @10], 
 [(-@2), @5, @8, @1, (-@11), (-@12)], 
 [@3, (-@6), @9, @11, @1, (-@13)], 
 [@4, @7, (-@10), @12, @13, @1]])

In [16]:
# Hydrostatics of Submerged Vehicles
# restoring forces

W = SX.sym('W')  # weight
B = SX.sym('B')  # buoyancy

thet = SX.sym('thet')  
phi = SX.sym('phi')  

g = SX(6, 1)

g[0,0] = (W - B)*sin(thet)
g[1,0] = -(W - B)*cos(thet)*sin(phi)
g[2,0] = -(W - B)*cos(thet)*cos(phi)
g[3,0] = -(y_g*W -y_b*B)*cos(thet)*cos(phi) + (z_g*W -z_b*B)*cos(thet)*sin(phi)
g[4,0] = (z_g*W -z_b*B)*sin(thet) + (x_g*W -x_b*B)*cos(thet)*cos(phi)
g[5,0] = -(x_g*W -x_b*B)*cos(thet)*sin(phi) - (y_g*W -y_b*B)*sin(thet)

#For neutrally buoyant vehicles W = B


In [ ]:
# Damping will be nonlinear and coupled for an underwater vehicle moving in 6 DOFs at high speed

# <xUabsU>	Quadratic drag in surge

# <yVabsV>	Quadratic drag in sway

# <zWabsW>	Quadratic drag in heave

# <kPabsP>	Quadratic drag in roll

# <mQabsQ>	Quadratic drag in pitch

# <nRabsR>	Quadratic drag in yaw



# vehicle is performing a noncoupled motion, we can assume a diagonal structure
X_u = SX.sym('X_u') # <xU>	Linear drag in surge
Y_v = SX.sym('Y_v') # <yV>	Linear drag in sway
Z_w = SX.sym('Z_w') # <zW>	Linear drag in heave
K_p = SX.sym('K_p') # <kP>	Linear drag in roll
M_q = SX.sym('M_q') # <mQ>	Linear drag in pitch
N_r = SX.sym('N_r') # <nR>	Linear drag in yaw

